In [1]:
import random
import shelve

from twnews.utils.memoize import load
from twnews.dataset.storage import TweetsStorage, NewsStorage
from twnews.dataset.url_resolver import UrlResolver
from twnews.dataset.texts import Tweet
from twnews import defaults
from twnews.utils.text_processors import Lemmatizer
from twnews.resolver import get_domain

In [2]:
corpus, _  = load('tf_idf_corpus')
corpus = set(corpus)
lemmatizer = Lemmatizer()
data_shelve = shelve.open(defaults.TWEETS_PATH)
resolver = UrlResolver()

2016-05-08 19:34:19.048693: Start of loading url_map from /home/avybornov/twnews_data_april/resolve_url_map.shelve
2016-05-08 19:34:19.050078: Url map successfully loaded


In [4]:
keys = data_shelve.keys()
random.shuffle(keys)

tweets = []
for i, key in enumerate(keys):
    tweet = Tweet(data_shelve[key])
    tweet.resolve_urls(resolver)
    tweets.append(tweet)
    
    if i % 10000 == 0:
        print '%.2f%%' % (i*100.0/len(keys))

0.00%
2.02%
4.04%
6.05%
8.07%
10.09%
12.11%
14.13%
16.14%
18.16%
20.18%
22.20%
24.22%
26.23%
28.25%
30.27%
32.29%
34.31%
36.32%
38.34%
40.36%
42.38%
44.39%
46.41%
48.43%
50.45%
52.47%
54.48%
56.50%
58.52%
60.54%
62.56%
64.57%
66.59%
68.61%
70.63%
72.65%
74.66%
76.68%
78.70%
80.72%
82.74%
84.75%
86.77%
88.79%
90.81%
92.83%
94.84%
96.86%
98.88%


In [5]:
news_storage = NewsStorage()
news_documents = news_storage.get_dataset_texts()

In [77]:
def _extract_entities(text):
    t = Text(text)
    t.language = 'ru'
    ent = t.entities
    res = []
    for e in ent:
        #print '\t',e
        for x in e:
            x_l = lemmatizer.lemmatize(x)
            #print '\t\t', x.lower()
            res.append(x_l)
    return set(res)
for x in _extract_entities('Владимир Путин блаблабла'): print x
#    pass

путин
владимир


In [83]:
from polyglot.text import Text
total_news_NE = set()
errors = 0
for i, news in enumerate(news_documents):
    text = news.title + '. ' + news.summary
    try:
        ents = _extract_entities(text)
    except Exception:
        errors += 1
        ents = set()
    news.entitites = ents
    #print ents
    #for x in ents: print '\t', x
    #ents = map(lambda x: x.tolower(), ents)
    #print total_news_NE
    #for x in ents: print '\t', x

    total_news_NE.update(ents)
    
    if i % 1000 == 0:
        print '%.2f%%' % (i*100.0/len(news_documents))

#     for x in ents:
#         print x
    
#     print ent

#     print news.summary

0.00%
7.29%
14.59%
21.88%
29.17%
36.47%
43.76%
51.05%
58.35%
65.64%
72.93%
80.23%
87.52%
94.81%


In [88]:
print errors
print len(total_news_NE)
for putin in ['Путин', u'Путин', u'путин']:
    print putin.__class__, putin,
    print putin in total_news_NE

# for x in list(total_news_NE)[:100]:
#     print x
print len(tweets)

36
9072
<type 'str'> Путин False
<type 'unicode'> Путин False
<type 'unicode'> путин True
495552


In [ ]:
already_marked = set([])
files = ['random_500/random_result', '']

In [138]:
bad_domain = set(['apps.facebook.com', 'ask.fm', 'twitter.com', 'apps.facebook.com', 'www.instagram.com', 'www.instagram.com', 'vk.cc'])
min_num_of_words_in_NE = 2
already_marked = load('already_marked')
def is_tweet_good(tweet):
    if tweet.retweet:
        return False
    
    if tweet.tweet_id in already_marked:
        return False
    
    for url in tweet.urls:
        domain = get_domain(url)
        if domain in bad_domain:
            return False
        
    text = tweet.text
    
#     for h in tweet.hastags:
#         text = text.replace(h, '')
    
    words = lemmatizer.split_text_to_lemmas(text)
    
    #print map(lambda x: x.__class__, words)
    #words = map(lambda x: x.decode('utf-8'), words)
#     if len(words) < min_num_of_words:
#         return False
    
    num_of_words_in_NE = sum(map(lambda x: x in total_news_NE, words))
    if num_of_words_in_NE < min_num_of_words_in_NE:
        return False
    tweet.num_of_words_in_NE = num_of_words_in_NE
    
    return True

def filtration(tweets):
    res = []
    for i, tweet in enumerate(tweets):
        if is_tweet_good(tweet):
            res.append(tweet)
        if i % 5000 == 0:
            print '%.2f%%' % (i*100.0/len(tweets))
    print len(tweets), len(res)
    return res

print tweets[0]
print tweets[0].urls
print is_tweet_good(tweets[0])

Станислав Белковский: Базовая идея Владимира Путина состоит в том, что Украина все равно развалится и рухнет 
['http://www.svoboda.org/content/transcript/27668832.html']
False


In [140]:
good_tweets = filtration(tweets[:50000])
print len(good_tweets)

0.00%
10.00%
20.00%
30.00%
40.00%
50.00%
60.00%
70.00%
80.00%
90.00%
50000 13666
13666


In [142]:
from collections import defaultdict
words_distribution = defaultdict(list)
for tweet in good_tweets:
    k = tweet.num_of_words_in_NE
    words_distribution[k].append(tweet.text)
   # print tweet
good_tweets_1 = filter(lambda x: x.num_of_words_in_NE >= 2, good_tweets)[:10000]    
print len(good_tweets_1)
# words_distribution = sorted(words_distribution.items(), key=lambda x: len(x[1]), reverse=True)
# for k, texts in words_distribution:
#     print k, len(texts)
#     for x in texts[:10]:
#         print '\t',x.replace('\n', ' ')[:80]

10000


In [143]:
# for tweet in tweets[:20]:
#     print tweet
#     good = is_tweet_good(tweet)
#     if good: print tweet.num_of_words_in_NE
#     #print '----'

In [144]:
from twnews.utils.memoize import dump

dump(good_tweets_1, 'tweet_filtered_versus_3')